In [1]:
from pyspark.sql import SparkSession
import os

In [3]:
local=False
if local:
    spark=SparkSession.builder.master("local[4]") \
                  .appName("aida_poc_etl").getOrCreate()
else:
    spark=SparkSession.builder \
                      .master("k8s://https://kubernetes.default.svc:443") \
                      .appName("aida_poc_etl") \
                      .config("spark.kubernetes.container.image",os.environ["IMAGE_NAME"]) \
                      .config("spark.kubernetes.authenticate.driver.serviceAccountName",os.environ['KUBERNETES_SERVICE_ACCOUNT']) \
                      .config("spark.kubernetes.namespace", os.environ['KUBERNETES_NAMESPACE']) \
                      .config("spark.executor.instances", "8") \
                      .config("spark.executor.memory","16g") \
                      .config('spark.jars.packages','org.postgresql:postgresql:42.2.24') \
                      .getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e0a85f61-9893-4915-a676-f8936f75b18d;1.0
	confs: [default]
	found org.postgresql#postgresql;42.2.24 in central
	found org.checkerframework#checker-qual;3.5.0 in central
downloading https://repo1.maven.org/maven2/org/postgresql/postgresql/42.2.24/postgresql-42.2.24.jar ...
	[SUCCESSFUL ] org.postgresql#postgresql;42.2.24!postgresql.jar (126ms)
downloading https://repo1.maven.org/maven2/org/checkerframework/checker-qual/3.5.0/checker-qual-3.5.0.jar ...
	[SUCCESSFUL ] org.checkerframework#checker-qual;3.5.0!checker-qual.jar (34ms)
:: resolution report :: resolve 839ms :: artifacts dl 166ms
	:: modules in use:
	org.checkerframework#checker-qual;3.5.0 from central in [default]
	org.postgresql#postgresql;42.2.24 from central in [default]
	-------------------------------

In [15]:
work_dir="s3a://projet-poc-aida/rp"
file_name="individus.csv"
file_path=f"{work_dir}/{file_name}"

# use option
df=spark.read\
    .option("header",True)\
    .option("inferSchema", True) \
    .option("delimiter",';') \
    .csv(path=file_path)

2022-04-27 08:54:07,198 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_38_29 !
2022-04-27 08:54:07,198 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_38_51 !
2022-04-27 08:54:07,198 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_38_11 !
2022-04-27 08:54:07,198 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_38_59 !
2022-04-27 08:54:07,199 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_38_68 !
2022-04-27 08:54:07,199 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_38_19 !
2022-04-27 08:54:07,199 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_38_34 !
2022-04-27 08:54:07,199 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_38_94 !
2022-04-27 08:54:07,199 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_38_44 !
2022-04-27 08:54:07,199 WARN

In [16]:
# df.cache()
df.show(5,truncate=False)

2022-04-27 08:55:56,666 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------------+---------------------+-----------------+--------------+-------------------+---------------+------------------+-------------+--------------+------------------+----+----------+-------+----------------+-------+---+---------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|region_residence|departement_residence|commune_residence|region_travail|departement_travail|commune_travail|commune_anterieure|commune_etude|pays_naissance|poids             |sexe|statut_pro|densite|recherche_emplo

In [17]:
df.count()

50000000

In [18]:
df.printSchema()

root
 |-- region_residence: integer (nullable = true)
 |-- departement_residence: string (nullable = true)
 |-- commune_residence: string (nullable = true)
 |-- region_travail: integer (nullable = true)
 |-- departement_travail: string (nullable = true)
 |-- commune_travail: string (nullable = true)
 |-- commune_anterieure: string (nullable = true)
 |-- commune_etude: string (nullable = true)
 |-- pays_naissance: integer (nullable = true)
 |-- poids: double (nullable = true)
 |-- sexe: integer (nullable = true)
 |-- statut_pro: string (nullable = true)
 |-- densite: integer (nullable = true)
 |-- recherche_emploi: integer (nullable = true)
 |-- diplome: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- variable00: long (nullable = true)
 |-- variable01: integer (nullable = true)
 |-- variable02: integer (nullable = true)
 |-- variable03: integer (nullable = true)
 |-- variable04: double (nullable = true)
 |-- variable05: double (nullable = true)
 |-- variable06: intege

In [19]:
partition_number=df.rdd.getNumPartitions()
print(f"Data partition number: {partition_number}")

Data partition number: 109


In [ ]:
output_file="individus_snappy_parquet"

output_path=f"{work_dir}/{output_file}"

df.write.mode("overwrite").parquet(output_path)

2022-04-27 08:56:57,817 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_38_85 !
2022-04-27 08:56:57,817 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_38_107 !
2022-04-27 08:56:57,817 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_38_16 !
2022-04-27 08:56:57,817 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_38_6 !
2022-04-27 08:56:57,817 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_38_62 !
2022-04-27 08:56:57,818 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_38_50 !
2022-04-27 08:56:57,818 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_38_38 !
2022-04-27 08:56:57,818 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_38_24 !
2022-04-27 08:56:57,818 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_38_96 !
2022-04-27 08:56:57,818 WARN